# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [ ]:
using Plots; plotly()

In [ ]:
@time begin
    
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
end

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [ ]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

In [ ]:
@time walk(1)

In [ ]:
@time walk(100)

In [ ]:
@time walk(1000)

## Draw a random walk

In [ ]:
function trajectory(numsteps=1000)

    pos = 0 
    positions = [pos]

    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
        push!(positions, pos)

    end
    
    positions
end


In [ ]:
numsteps = 1000
plot(1:numsteps, trajectory(numsteps))

### Scalability

In [ ]:
using Interact

In [ ]:
@manipulate for k in 3:9
    @elapsed walk(10^k)
end

In [ ]:
plot(3:9, [@elapsed walk(10^k) for k = 3:9])

## Add parallelism

In [ ]:
addprocs(2)

In [ ]:
@everywhere using DistributedArrays

In [ ]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

In [ ]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end
serialwalkers = collect(1:numwalkers)


In [ ]:
parallelwalkers = distribute(serialwalkers)

In [ ]:
parallelwalkers.indexes

In [ ]:
typeof(parallelwalkers)

### Benchmarking

In [ ]:
using BenchmarkTools

In serial:

In [ ]:
@benchmark map(_ -> walk(numsteps), serialwalkers)

In parallel:

In [ ]:
@benchmark positions = map( _ -> walk(numsteps), parallelwalkers)